# 데이터 셋

* 데이터 출처 : kaggle의 olist sellers dataset을 사용했습니다 
  * url : https://www.kaggle.com/olistbr/brazilian-ecommerce?select=olist_sellers_dataset.csv

# 분석 목표 
* 현재 상황을 파악하여 매출 개선 계획 
* 2023년 프로모션 진행은 언제, 어느 카테고리에서 진행하는 것이 좋을까?

-----
가설 : 
* 우리 서비스의 매출 상황은 점점 성장하고 있을 것이다.
    * 전체적인 구매자 수가 증가했다
    * ARPPU가 증가했다
    * 장바구니 크기가 증가했다
    * 연간 구입 건수가 증가했다

* 분기별 매출에서는 4분기 매출이 가장 높을 것이다.
    * 연말과 블랙 프라이데이가 집중되어 있는 4분기 매출이 가장 높을 것.

* 주말보다 평일에서의 매출이 더 많이 나올 것이다 
    * 주말에는 외부 활동이 많아지기 때문에 평일에 구매가 더 많이 일어날 것

# 데이터 정제 

## 분석 시 필요한 라이브러리 

In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import style 
import seaborn as sns
from datetime import datetime,timedelta
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', 100)

## 데이터 불러오기 


In [5]:
customers = pd.read_csv("c:/data/olist/olist_customers_dataset.csv")
order_items = pd.read_csv('c:/data/olist/olist_order_items_dataset.csv')
order_payments = pd.read_csv("c:/data/olist/olist_order_payments_dataset.csv")
products = pd.read_csv('c:/data/olist/olist_products_dataset.csv')
category_translation = pd.read_csv("c:/data/olist/product_category_name_translation.csv")
orders = pd.read_csv('c:/data/olist/olist_orders_dataset.csv')


In [8]:
# displaying data shape 
dataset = {
    'Customers' : customers,
    'Order Items' : order_items,
    'Payments' : order_payments, 
    'Orders' : orders,
    'Products' : products,
    'Translation' : category_translation
}

# x(총 row수), y(컬럼 수)
for x,y in dataset.items():
    print(f'{x}', (list(y.shape)))

Customers [99441, 5]
Order Items [112650, 7]
Payments [103886, 5]
Orders [99441, 8]
Products [32951, 9]
Translation [71, 2]


In [10]:
# displaying dataset column names 
for x, y in dataset.items():
    print(f'{x}', f'{list(y.columns)}\n')

Customers ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']

Order Items ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']

Payments ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']

Orders ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

Products ['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

Translation ['product_category_name', 'product_category_name_english']



In [12]:
# checking for null values in dataset 
for x, y in dataset.items():
    print(f'{x}: {y.isnull().any().any()}')

Customers: False
Order Items: False
Payments: False
Orders: True
Products: True
Translation: False


In [33]:
# taking count for dataset with missing values 
for x, y in dataset.items():
    if y.isnull().any().any():
        print(f'{x}', (list(y.shape)),'\n')
        print(f'{y.isnull().sum()}\n')

Orders [99441, 8] 

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Products [32951, 9] 

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64



## creating master dataframe 

**분석의 편의성을 위해 결측치는 drop 처리 하였습니다.** 

### Translate Product Name Into English

In [15]:
products_en = pd.merge(products, category_translation, how = 'outer', on = 'product_category_name')
products_en

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumaria,39.0,346.0,2.0,400.0,27.0,5.0,20.0,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumaria,52.0,150.0,1.0,422.0,21.0,16.0,18.0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumaria,49.0,460.0,2.0,267.0,17.0,13.0,17.0,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumaria,56.0,733.0,3.0,377.0,18.0,13.0,15.0,perfumery
...,...,...,...,...,...,...,...,...,...,...
32946,13b25797ad1e6d6c7cd3cbeb35f8ee7a,moveis_colchao_e_estofado,44.0,348.0,1.0,18250.0,25.0,25.0,25.0,furniture_mattress_and_upholstery
32947,dc36a7859b743d8610a2bbbaea26ece9,moveis_colchao_e_estofado,29.0,2073.0,2.0,2500.0,41.0,21.0,21.0,furniture_mattress_and_upholstery
32948,107fde0930956120d1e13dd1062fbb46,moveis_colchao_e_estofado,51.0,418.0,1.0,30000.0,60.0,73.0,60.0,furniture_mattress_and_upholstery
32949,726b4e18f00255e2e63491bcba3f60b8,moveis_colchao_e_estofado,41.0,866.0,1.0,1700.0,70.0,10.0,100.0,furniture_mattress_and_upholstery


In [16]:
# 어떤 카테고리의 제품 이름이 번역되지 않았는지 찾아봅니다. 
set(products['product_category_name'].unique()) - set(category_translation['product_category_name'].unique())

{nan, 'pc_gamer', 'portateis_cozinha_e_preparadores_de_alimentos'}

In [19]:
# 번역되지 않은 카테고리는 수기로 변경함 
products_en['product_category_name_english'][products_en['product_category_name'] == 'pc_gamer'] = 'pc_gamer'
products_en['product_category_name_english'][products_en['product_category_name'] == 'portateis_cozinha_e_preparadores_de_alimentos'] = 'portable_kitchen_and_food_preparation'
products_en['product_category_name_english'][products_en['product_category_name'] == 'nan'] = 'unclassified'

In [22]:
# copy해두기 
products = products_en.copy()

### merge dataset 

In [23]:
# creating master dataframe 
df = pd.merge(orders, order_items, on = 'order_id', how = 'outer')
df = pd.merge(df, order_payments, on = 'order_id', how = 'outer')
df = pd.merge(df, products, on = 'product_id', how = 'outer')
df = pd.merge(df, customers, on = 'customer_id', how = 'outer')
print(df.shape)

(118434, 31)


### converting data columns to datetime

In [25]:
# converting date columns to datetime
date_columns = ['shipping_limit_date', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')

## Preprocessing Some columns

### cleaning up name columns

In [32]:
df['customer_city'] = df['customer_city'].str.title() # 대문자 변경 | 보기 쉬워짐
df['payment_type'] = df['payment_type'].str.replace('_',' ').str.title() #언더바를 띄어쓰기로 변환

df[['customer_city','payment_type']]

,customer_city,payment_type
0,Sao Paulo,Credit Card
1,Sao Paulo,Voucher
2,Sao Paulo,Voucher
3,Sao Paulo,Credit Card
4,Sao Paulo,Credit Card
...,...,...
118429,Sao Paulo,Credit Card
118430,Sao Paulo,Credit Card
118431,Campo Do Tenente,Credit Card
118432,Sao Jose Dos Campos,Credit Card


### engineering new/essential columns